In [3]:
#Keep in mind none of the BLP outputs are shipped within the repo.

#nucl_gb.accession2taxid, nodesRED.dmp & namesRED.dmp are also not in here (download from NCBI taxdump).

In [ ]:
import os
import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from progressbar import *
import csv

In [4]:
Nodes = []
with open('../data/accessions_phylo') as f:
    for line in f:
        Nodes.append(line.strip())

In [5]:
#Download the prot 2 accession from ncbi taxonomy, unzip: file = prot.accession2taxid
#ProgressBar().register()
dask_df = dd.read_csv('../data/nucl_gb.accession2taxid', header=0, sep='\t')
dask_df.npartitions
taxdask = dask_df[dask_df['accession'].isin(Nodes)]
taxlist = taxdask.compute()

In [6]:
del taxlist['accession.version']
del taxlist['gi']
#Check for missing values:
for i in Nodes:
    if i not in list(taxlist['accession']):
        print(i)

Hubei
BP46
BP47
BP49
MK760677
MK760709
MK760727
MK760728
MK760739
MK760764


In [7]:
taxnes = []
for index, row in taxlist.iterrows():
    taxnes.append([row['accession'], row['taxid']])

In [16]:
#Download the taxdump.tar.gz from NCBI taxonomy and unzip. 
#Enumerate taxids until root.
widgets = ['Enumerating taxpaths: ', Percentage(), ' ', Bar(marker='0',left='[',right=']'),' ', ETA(), ' ', FileTransferSpeed()]
pbar = ProgressBar(widgets=widgets, maxval=len(taxnes))
nodes = pd.read_csv("../data/nodesRED.dmp",delimiter='\t',encoding='utf-8')
taxpathlist = []
pbar.start()
count=0
for i in taxnes:
    count += 1
    name = i[0]
    tax = i[1]
    templist = [name,tax]
    while tax != 1:
        newnod = nodes.loc[nodes['tax'] == int(tax)]
        try:
            tax = newnod.iat[0,1]
        except:
            tax = 1
        templist.append(tax)
    taxpathlist.append(templist)
    pbar.update(count)
pbar.finish()

Enumerating taxpaths: 100% [000000000000000000000000] Time: 0:00:29  67.19  B/s


In [17]:
taxpathlist2 = []
for i in taxpathlist:
    templis = [i[0]] + i[1:][::-1]
    if 10239 in templis:
        taxpathlist2.append(templis)

In [18]:
#Before naming them, get the 'rank' of each of the tax ids.
nodes = pd.read_csv("../data/nodesRED.dmp",delimiter='\t',encoding='utf-8')
virdic = {}
for i in taxpathlist2:
    taxdic = {}
    for j in i:
        if j == i[0]:
            name = str(j)
        else:
            tax = int(j)
            rank = nodes[nodes['tax'] == tax].iat[0,2]
            taxdic[rank] = tax
    virdic[name] = taxdic

In [19]:
#Change taxID numbers into names
names = pd.read_csv("../data/namesRED.dmp",delimiter='\t',encoding='utf-8')
namedic = {}
for i in virdic:
    name = str(i)
    tempdic = {}
    for j in virdic[i]:
        nam = names[names['tax'] == int(virdic[i][j])].iat[0,1]
        tempdic[j] = nam
    namedic[i] = tempdic

In [20]:
namedf = pd.DataFrame(namedic)
namedf = namedf.T
namedf = namedf.fillna('Unclassified')
namedf.head()

,no rank,superkingdom,phylum,subphylum,class,order,family,genus,species,subfamily
AB009601,RNA viruses,Vira,Negarnaviricota,Haploviricotina,Monjiviricetes,Mononegavirales,Rhabdoviridae,Lyssavirus,Rabies lyssavirus,Unclassified
AB011257,RNA viruses,Vira,Negarnaviricota,Haploviricotina,Monjiviricetes,Mononegavirales,Rhabdoviridae,Nucleorhabdovirus,RTYV,Unclassified
AB025903,Helicobasidium mompa dsRNA mycovirus,Vira,Unclassified,Unclassified,Unclassified,Unclassified,Partitiviridae,Alphacryptovirus,Helicobasidium mompa partitivirus V70,Unclassified
AB045371,RNA viruses,Vira,Unclassified,Unclassified,Unclassified,Tymovirales,Betaflexiviridae,Foveavirus,ASPV,Quinvirinae
AB070959,Kakugo virus,Vira,Unclassified,Unclassified,Unclassified,Picornavirales,Iflaviridae,Infectious flachery-like viruses,DWV,Unclassified


In [21]:
################### Bunya ###################
bunyarefdic = {}
bunyanodes = []
bunyacontigs = []
with open('../data/phylo_nodes/bunya_treenames') as f:
    for line in f:
        if 'NODE' in line.strip():
            bunyanodes.append(line.strip())
        elif 'Contig' in line.strip():
            bunyacontigs.append(line.strip())
        else:
            string1 = '_'.join(line.strip().split('_')[:-1])
            string2 = string1.split('.')[0]
            bunyarefdic[line.strip()] = string2

In [22]:
Feraviridae = list(namedf[namedf.index.isin(list(bunyarefdic.values()))][namedf[namedf.index.isin(list(bunyarefdic.values()))]['family'] == 'Feraviridae'].index)
unclas_bunya = list(namedf[namedf.index.isin(list(bunyarefdic.values()))][namedf[namedf.index.isin(list(bunyarefdic.values()))]['family'] == 'Unclassified'].index)
bunyaviridae = list(namedf[namedf.index.isin(list(bunyarefdic.values()))][namedf[namedf.index.isin(list(bunyarefdic.values()))]['family'] == 'Bunyaviridae'].index)

#Feraviridae:
#for i in bunyarefdic:
#    if bunyarefdic[i] in Feraviridae:
#        print(i)


##Unclassified:
#for i in bunyarefdic:
#    if bunyarefdic[i] in unclas_bunya:
#        print(i)

##Bunyaviridae
#for i in bunyarefdic:
#    if bunyarefdic[i] in bunyaviridae:
#        print(i)

In [24]:
################### Orthomyxo_bee ###################
orthorefdic = {}
orthonodes = []
with open('../data/phylo_nodes/orthomyxobee_treenames') as f:
    for line in f:
        if 'BP46' in line.strip() or 'BP47' in line.strip() or 'BP49' in line.strip():
            orthonodes.append(line.strip())
        else:
            orthorefdic[line.strip()] = line.strip().split('_')[0]

In [25]:
gammainfluenza = list(namedf[namedf.index.isin(list(orthorefdic.values()))][namedf[namedf.index.isin(list(orthorefdic.values()))]['genus'] == 'Gammainfluenzavirus'].index)
alphainfluenza = list(namedf[namedf.index.isin(list(orthorefdic.values()))][namedf[namedf.index.isin(list(orthorefdic.values()))]['genus'] == 'Alphainfluenzavirus'].index)
betainfluenza = list(namedf[namedf.index.isin(list(orthorefdic.values()))][namedf[namedf.index.isin(list(orthorefdic.values()))]['genus'] == 'Betainfluenzavirus'].index)
thogotolike = list(namedf[namedf.index.isin(list(orthorefdic.values()))][namedf[namedf.index.isin(list(orthorefdic.values()))]['genus'] == 'Thogoto-like viruses'].index)
quaranja = list(namedf[namedf.index.isin(list(orthorefdic.values()))][namedf[namedf.index.isin(list(orthorefdic.values()))]['genus'] == 'Quaranjavirus'].index)
isa = list(namedf[namedf.index.isin(list(orthorefdic.values()))][namedf[namedf.index.isin(list(orthorefdic.values()))]['genus'] == 'Isavirus'].index)
unclassified = list(namedf[namedf.index.isin(list(orthorefdic.values()))][namedf[namedf.index.isin(list(orthorefdic.values()))]['genus'] == 'Unclassified'].index)


##gammainfluenza:
#for i in orthorefdic:
#    if orthorefdic[i] in gammainfluenza:
#        print(i)

##alphainfluenza:
#for i in orthorefdic:
#    if orthorefdic[i] in alphainfluenza:
#        print(i)

##betainfluenza:
#for i in orthorefdic:
#    if orthorefdic[i] in betainfluenza:
#        print(i)

##thogotolike:
#for i in orthorefdic:
#    if orthorefdic[i] in thogotolike:
#        print(i)

##quaranja:
#for i in orthorefdic:
#    if orthorefdic[i] in quaranja:
#        print(i)

#isa:
#for i in orthorefdic:
#    if orthorefdic[i] in isa:
#        print(i)

#unclassified:
#for i in orthorefdic:
#    if orthorefdic[i] in unclassified:
#        print(i)

In [26]:
################### Orthomyxo_sra ###################
orthosrarefdic = {}
orthosranodes = []
orthosracontigs = []
with open('../data/phylo_nodes/orthomyxosra_treenames') as f:
    for line in f:
        if 'NODE' in line.strip():
            orthosranodes.append(line.strip())
        elif 'Contig' in line.strip():
            orthosracontigs.append(line.strip())
        else:
            orthosrarefdic[line.strip()] = line.strip().split('_')[0].split('.')[0]

In [27]:
deltainfluenza = list(namedf[namedf.index.isin(list(orthosrarefdic.values()))][namedf[namedf.index.isin(list(orthosrarefdic.values()))]['genus'] == 'Deltainfluenzavirus'].index)
alphainfluenza = list(namedf[namedf.index.isin(list(orthosrarefdic.values()))][namedf[namedf.index.isin(list(orthosrarefdic.values()))]['genus'] == 'Alphainfluenzavirus'].index)
betainfluenza = list(namedf[namedf.index.isin(list(orthosrarefdic.values()))][namedf[namedf.index.isin(list(orthosrarefdic.values()))]['genus'] == 'Betainfluenzavirus'].index)
thogotolike = list(namedf[namedf.index.isin(list(orthosrarefdic.values()))][namedf[namedf.index.isin(list(orthosrarefdic.values()))]['genus'] == 'Thogoto-like viruses'].index)
quaranja = list(namedf[namedf.index.isin(list(orthosrarefdic.values()))][namedf[namedf.index.isin(list(orthosrarefdic.values()))]['genus'] == 'Quaranjavirus'].index)
unclassified = list(namedf[namedf.index.isin(list(orthosrarefdic.values()))][namedf[namedf.index.isin(list(orthosrarefdic.values()))]['genus'] == 'Unclassified'].index)


##deltainfluenza:
#for i in orthosrarefdic:
#    if orthosrarefdic[i] in deltainfluenza:
#        print(i)

##alphainfluenza:
#for i in orthosrarefdic:
#    if orthosrarefdic[i] in alphainfluenza:
#        print(i)

##betainfluenza:
#for i in orthosrarefdic:
#    if orthosrarefdic[i] in betainfluenza:
#        print(i)

##thogotolike:
#for i in orthosrarefdic:
#    if orthosrarefdic[i] in thogotolike:
#        print(i)

##quaranja:
#for i in orthosrarefdic:
#    if orthosrarefdic[i] in quaranja:
#        print(i)

##unclassified:
#for i in orthosrarefdic:
#    if orthosrarefdic[i] in unclassified:
#        print(i)

In [29]:
################### SinaiNS ###################
##### This could be a sinai, but the non-structural is from bastro (recombinant), so no sinai refs here.
sinainsrefdic = {}
sinainsnodes = []
with open('../data/phylo_nodes/sinains_treenames') as f:
    for line in f:
        if 'NODE' in line.strip():
            sinainsnodes.append(line.strip())
        else:
            sinainsrefdic[line.strip()] = '_'.join(line.strip().split('_')[:-1]).split('.')[0]

In [30]:
alphatetraviridae = list(namedf[namedf.index.isin(list(sinainsrefdic.values()))][namedf[namedf.index.isin(list(sinainsrefdic.values()))]['family'] == 'Alphatetraviridae'].index)
Astroviridae = list(namedf[namedf.index.isin(list(sinainsrefdic.values()))][namedf[namedf.index.isin(list(sinainsrefdic.values()))]['family'] == 'Astroviridae'].index)
Hepeviridae = list(namedf[namedf.index.isin(list(sinainsrefdic.values()))][namedf[namedf.index.isin(list(sinainsrefdic.values()))]['family'] == 'Hepeviridae'].index)
Bromoviridae = list(namedf[namedf.index.isin(list(sinainsrefdic.values()))][namedf[namedf.index.isin(list(sinainsrefdic.values()))]['family'] == 'Bromoviridae'].index)

##alphatetraviridae:
#for i in sinainsrefdic:
#    if sinainsrefdic[i] in alphatetraviridae:
#        print(i)

##Astroviridae:
#for i in sinainsrefdic:
#    if sinainsrefdic[i] in Astroviridae:
#        print(i)

##Hepeviridae:
#for i in sinainsrefdic:
#    if sinainsrefdic[i] in Hepeviridae:
#        print(i)

##Bromoviridae:
#for i in sinainsrefdic:
#    if sinainsrefdic[i] in Bromoviridae:
#        print(i)

In [31]:
################### SinaiS ###################
sinaisrefdic = {}
sinaisnodes = []
sinaiscontigs = []
with open('../data/phylo_nodes/sinais_treenames') as f:
    for line in f:
        if 'NODE' in line.strip():
            sinaisnodes.append(line.strip())
        if 'Contig' in line.strip():
            sinaiscontigs.append(line.strip())
        else:
            sinaisrefdic[line.strip()] = '_'.join(line.strip().split('_')[:-1]).split('.')[0]

In [32]:
sinai1 = list(namedf[namedf.index.isin(list(sinaisrefdic.values()))][namedf[namedf.index.isin(list(sinaisrefdic.values()))]['species'] == 'Lake Sinai virus 1'].index)
sinai2 = list(namedf[namedf.index.isin(list(sinaisrefdic.values()))][namedf[namedf.index.isin(list(sinaisrefdic.values()))]['species'] == 'Lake Sinai virus 2'].index)
sinai = list(namedf[namedf.index.isin(list(sinaisrefdic.values()))][namedf[namedf.index.isin(list(sinaisrefdic.values()))]['species'] == 'Lake Sinai virus'].index)
sinaiTO = list(namedf[namedf.index.isin(list(sinaisrefdic.values()))][namedf[namedf.index.isin(list(sinaisrefdic.values()))]['species'] == 'Lake Sinai Virus TO'].index)
sinaiNE = list(namedf[namedf.index.isin(list(sinaisrefdic.values()))][namedf[namedf.index.isin(list(sinaisrefdic.values()))]['species'] == 'Lake Sinai Virus NE'].index)
sinaiSA1 = list(namedf[namedf.index.isin(list(sinaisrefdic.values()))][namedf[namedf.index.isin(list(sinaisrefdic.values()))]['species'] == 'Lake Sinai Virus SA1'].index)
sinaiSA2 = list(namedf[namedf.index.isin(list(sinaisrefdic.values()))][namedf[namedf.index.isin(list(sinaisrefdic.values()))]['species'] == 'Lake Sinai Virus SA2'].index)
sinai3 = list(namedf[namedf.index.isin(list(sinaisrefdic.values()))][namedf[namedf.index.isin(list(sinaisrefdic.values()))]['species'] == 'Lake Sinai virus 3'].index)

##sinai1:
#for i in sinaisrefdic:
#    if sinaisrefdic[i] in sinai1:
#        print(i)

##sinai2:
#for i in sinaisrefdic:
#    if sinaisrefdic[i] in sinai2:
#        print(i)

##sinai:
#for i in sinaisrefdic:
#    if sinaisrefdic[i] in sinai:
#        print(i)

##sinaiTO:
#for i in sinaisrefdic:
#    if sinaisrefdic[i] in sinaiTO:
#        print(i)

##sinaiNE:
#for i in sinaisrefdic:
#    if sinaisrefdic[i] in sinaiNE:
#        print(i)

##sinaiSA1:
#for i in sinaisrefdic:
#    if sinaisrefdic[i] in sinaiSA1:
#        print(i)

##sinaiSA2:
#for i in sinaisrefdic:
#    if sinaisrefdic[i] in sinaiSA2:
#        print(i)

##sinai3:
#for i in sinaisrefdic:
#    if sinaisrefdic[i] in sinai3:
#        print(i)

In [33]:
################### Partiti ###################
partitirefdic = {}
partitinodes = []
partiticontigs = []
with open('../data/phylo_nodes/partiti_treenames') as f:
    for line in f:
        if 'NODE' in line.strip():
            partitinodes.append(line.strip())
        if 'Contig' in line.strip():
            partiticontigs.append(line.strip())
        else:
            partitirefdic[line.strip()] = '_'.join(line.strip().split('_')[:-1]).split('.')[0]

In [34]:
Alphacryptovirus = list(namedf[namedf.index.isin(list(partitirefdic.values()))][namedf[namedf.index.isin(list(partitirefdic.values()))]['genus'] == 'Alphacryptovirus'].index)
Betacryptovirus = list(namedf[namedf.index.isin(list(partitirefdic.values()))][namedf[namedf.index.isin(list(partitirefdic.values()))]['genus'] == 'Betacryptovirus'].index)
Gammapartitivirus = list(namedf[namedf.index.isin(list(partitirefdic.values()))][namedf[namedf.index.isin(list(partitirefdic.values()))]['genus'] == 'Gammapartitivirus'].index)
Unclassified = list(namedf[namedf.index.isin(list(partitirefdic.values()))][namedf[namedf.index.isin(list(partitirefdic.values()))]['genus'] == 'Unclassified'].index)

##Alphacryptovirus:
#for i in partitirefdic:
#    if partitirefdic[i] in Alphacryptovirus:
#        print(i)

##Betacryptovirus:
#for i in partitirefdic:
#    if partitirefdic[i] in Betacryptovirus:
#        print(i)

##Gammapartitivirus:
#for i in partitirefdic:
#    if partitirefdic[i] in Gammapartitivirus:
#        print(i)

##Unclassified:
#for i in partitirefdic:
#    if partitirefdic[i] in Unclassified:
#        print(i)

In [35]:
################### Tymo ###################
tymorefdic = {}
tymonodes = []
tymocontigs = []
with open('../data/phylo_nodes/tymo_refseqtreenames') as f:
    for line in f:
        if 'NODE' in line.strip():
            tymonodes.append(line.strip())
        if 'Contig' in line.strip():
            tymocontigs.append(line.strip())
        else:
            tymorefdic[line.strip()] = '_'.join(line.strip().split('_')[:-1]).split('.')[0]

In [36]:
tymoviridae = list(namedf[namedf.index.isin(list(tymorefdic.values()))][namedf[namedf.index.isin(list(tymorefdic.values()))]['family'] == 'Tymoviridae'].index)
betaflex = list(namedf[namedf.index.isin(list(tymorefdic.values()))][namedf[namedf.index.isin(list(tymorefdic.values()))]['family'] == 'Betaflexiviridae'].index)
alphaflex = list(namedf[namedf.index.isin(list(tymorefdic.values()))][namedf[namedf.index.isin(list(tymorefdic.values()))]['family'] == 'Alphaflexiviridae'].index)

##tymoviridae:
#for i in tymorefdic:
#    if tymorefdic[i] in tymoviridae:
#        print(i)

##betaflex:
#for i in tymorefdic:
#    if tymorefdic[i] in betaflex:
#        print(i)

##alphaflex:
#for i in tymorefdic:
#    if tymorefdic[i] in alphaflex:
#        print(i)

In [37]:
################### Toti ###################
totirefdic = {}
totinodes = []
toticontigs = []
with open('../data/phylo_nodes/toti_treenames') as f:
    for line in f:
        if 'NODE' in line.strip():
            totinodes.append(line.strip())
        if 'Contig' in line.strip():
            toticontigs.append(line.strip())
        else:
            totirefdic[line.strip()] = '_'.join(line.strip().split('_')[:-1]).split('.')[0]

In [38]:
victori = list(namedf[namedf.index.isin(list(totirefdic.values()))][namedf[namedf.index.isin(list(totirefdic.values()))]['genus'] == 'Victorivirus'].index)
toti = list(namedf[namedf.index.isin(list(totirefdic.values()))][namedf[namedf.index.isin(list(totirefdic.values()))]['genus'] == 'Totivirus'].index)
unclas = list(namedf[namedf.index.isin(list(totirefdic.values()))][namedf[namedf.index.isin(list(totirefdic.values()))]['genus'] == 'Unclassified'].index)


##victori:
#for i in totirefdic:
#    if totirefdic[i] in victori:
#        print(i)

##toti:
#for i in totirefdic:
#    if totirefdic[i] in toti:
#        print(i)

##unclas:
#for i in totirefdic:
#    if totirefdic[i] in unclas:
#        print(i)

In [39]:
################### Picorna ###################
picornarefdic = {}
picornanodes = []
picornacontigs = []
with open('../data/phylo_nodes/picorna_treenames') as f:
    for line in f:
        if 'NODE' in line.strip():
            picornanodes.append(line.strip())
        if 'Contig' in line.strip():
            picornacontigs.append(line.strip())
        else:
            picornarefdic[line.strip()] = '_'.join(line.strip().split('_')[:-1]).split('.')[0]

In [40]:
ifla = list(namedf[namedf.index.isin(list(picornarefdic.values()))][namedf[namedf.index.isin(list(picornarefdic.values()))]['family'] == 'Iflaviridae'].index)
como = list(namedf[namedf.index.isin(list(picornarefdic.values()))][namedf[namedf.index.isin(list(picornarefdic.values()))]['family'] == 'Comoviridae'].index)
dicistro = list(namedf[namedf.index.isin(list(picornarefdic.values()))][namedf[namedf.index.isin(list(picornarefdic.values()))]['family'] == 'Dicistroviridae'].index)
polycipi = list(namedf[namedf.index.isin(list(picornarefdic.values()))][namedf[namedf.index.isin(list(picornarefdic.values()))]['family'] == 'Polycipiviridae'].index)
marna = list(namedf[namedf.index.isin(list(picornarefdic.values()))][namedf[namedf.index.isin(list(picornarefdic.values()))]['family'] == 'Marnaviridae'].index)
picorna = list(namedf[namedf.index.isin(list(picornarefdic.values()))][namedf[namedf.index.isin(list(picornarefdic.values()))]['family'] == 'Picornaviridae'].index)
unclassified = list(namedf[namedf.index.isin(list(picornarefdic.values()))][namedf[namedf.index.isin(list(picornarefdic.values()))]['family'] == 'Unclassified'].index)


##ifla:
#for i in picornarefdic:
#    if picornarefdic[i] in ifla:
#        print(i)

##como:
#for i in picornarefdic:
#    if picornarefdic[i] in como:
#        print(i)

##dicistro:
#for i in picornarefdic:
#    if picornarefdic[i] in dicistro:
#        print(i)

##polycipi:
#for i in picornarefdic:
#    if picornarefdic[i] in polycipi:
#        print(i)

##marna:
#for i in picornarefdic:
#    if picornarefdic[i] in marna:
#        print(i)

##picorna:
#for i in picornarefdic:
#    if picornarefdic[i] in picorna:
#        print(i)

##unclass:
#for i in picornarefdic:
#    if picornarefdic[i] in unclassified:
#        print(i)

In [41]:
################### Rhabdo ###################
rhabdorefdic = {}
rhabdonodes = []
rhabdocontigs = []
with open('../data/phylo_nodes/rhabdo_treenames') as f:
    for line in f:
        if 'NODE' in line.strip():
            rhabdonodes.append(line.strip())
        if 'Contig' in line.strip():
            rhabdocontigs.append(line.strip())
        else:
            rhabdorefdic[line.strip()] = '_'.join(line.strip().split('_')[:-1]).split('.')[0]

In [43]:
rhabdo = list(namedf[namedf.index.isin(list(rhabdorefdic.values()))][namedf[namedf.index.isin(list(rhabdorefdic.values()))]['family'] == 'Rhabdoviridae'].index)
nyami = list(namedf[namedf.index.isin(list(rhabdorefdic.values()))][namedf[namedf.index.isin(list(rhabdorefdic.values()))]['family'] == 'Nyamiviridae'].index)
lispi = list(namedf[namedf.index.isin(list(rhabdorefdic.values()))][namedf[namedf.index.isin(list(rhabdorefdic.values()))]['family'] == 'Lispiviridae'].index)
xinmo = list(namedf[namedf.index.isin(list(rhabdorefdic.values()))][namedf[namedf.index.isin(list(rhabdorefdic.values()))]['family'] == 'Xinmoviridae'].index)
Mymona = list(namedf[namedf.index.isin(list(rhabdorefdic.values()))][namedf[namedf.index.isin(list(rhabdorefdic.values()))]['family'] == 'Mymonaviridae'].index)
arto = list(namedf[namedf.index.isin(list(rhabdorefdic.values()))][namedf[namedf.index.isin(list(rhabdorefdic.values()))]['family'] == 'Artoviridae'].index)
polycipi = list(namedf[namedf.index.isin(list(rhabdorefdic.values()))][namedf[namedf.index.isin(list(rhabdorefdic.values()))]['family'] == 'Polycipiviridae'].index)
narna = list(namedf[namedf.index.isin(list(rhabdorefdic.values()))][namedf[namedf.index.isin(list(rhabdorefdic.values()))]['family'] == 'Narnaviridae'].index)
unclassified = list(namedf[namedf.index.isin(list(rhabdorefdic.values()))][namedf[namedf.index.isin(list(rhabdorefdic.values()))]['family'] == 'Unclassified'].index)


##rhabdo:
#for i in rhabdorefdic:
#    if rhabdorefdic[i] in rhabdo:
#        print(i)

##nyami:
#for i in rhabdorefdic:
#    if rhabdorefdic[i] in nyami:
#        print(i)

##lispi:
#for i in rhabdorefdic:
#    if rhabdorefdic[i] in lispi:
#        print(i)

##xinmo:
#for i in rhabdorefdic:
#    if rhabdorefdic[i] in xinmo:
#        print(i)

##Mymona:
#for i in rhabdorefdic:
#    if rhabdorefdic[i] in Mymona:
#       print(i)

##arto:
#for i in rhabdorefdic:
#    if rhabdorefdic[i] in arto:
#        print(i)

##polycipi:
#for i in rhabdorefdic:
#    if rhabdorefdic[i] in polycipi:
#        print(i)
        
##narna:
#for i in rhabdorefdic:
#    if rhabdorefdic[i] in narna:
#        print(i)

##unclass:
#for i in rhabdorefdic:
#    if rhabdorefdic[i] in unclassified:
#        print(i)